In [ ]:
%pip install gradio

In [ ]:


import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio as gr

# Load your fine-tuned model and tokenizer
model_path = "./trained_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
model.eval()

def generate_tweet(prompt, max_length=50):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_length=max_length,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7,
            pad_token_id=tokenizer.pad_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

tweet_interface = gr.Interface(
    fn=generate_tweet,
    inputs=[
        gr.Textbox(lines=2, label="Prompt", placeholder="Enter the start of your tweet..."),
        gr.Slider(20, 100, value=50, step=1, label="Max Length")
    ],
    outputs=gr.Textbox(label="Generated Tweet"),
    title="AI Tweet Generator",
    description="Enter a prompt and let the fine-tuned GPT-2 model generate a tweet for you!"
)

tweet_interface.launch()
